In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/dalibra/MMD.git

fatal: destination path 'MMD' already exists and is not an empty directory.


In [3]:
cd MMD

/content/MMD


In [4]:
!pip install wandb -q

In [5]:
!pip install theano -q

In [6]:
!sed -i 's/blas_info = np.distutils.__config__.blas_opt_info/blas_info = np.distutils.__config__.blas_ilp64_opt_info/' "/usr/local/lib/python3.10/dist-packages/theano/configdefaults.py"
!sed -n '1284p' "/usr/local/lib/python3.10/dist-packages/theano/configdefaults.py"

            blas_info = np.distutils.__config__.blas_ilp64_opt_info


In [7]:
# !python Example_VAE.py -s mnist.npy

# Libraries

In [8]:
import json

In [9]:
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import wandb
!wandb login 1b8e8dc9dcf1a34397a04197c4826d3fe7441dae

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
import VariationalAutoencoder
import numpy as np
import scipy as sp
import time,os
import _pickle as cPickle
import gzip,copy,pickle
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from mmd import MMD_3_Sample_Test

# Data loading

In [11]:
print("Loading MNIST data")
#Retrieved from: http://deeplearning.net/data/mnist/mnist.pkl.gz

f = gzip.open('mnist.pkl.gz', 'rb')
(x_train, t_train), (x_valid, t_valid), (x_test, t_test)  = cPickle.load(f, encoding='latin1')
f.close()

print(x_train.shape, t_train.shape, x_valid.shape, t_valid.shape, x_test.shape, t_test.shape)
print(x_train.dtype)

Loading MNIST data
(50000, 784) (50000,) (10000, 784) (10000,) (10000, 784) (10000,)
float32


In [12]:
x_train=(x_train>0).astype('float')
x_valid=(x_valid>0).astype('float')
x_test=(x_test>0).astype('float')

# Train config

In [13]:
data=x_train
samp_size=5000
verbose=True
runs=10
# runs=1
ratios=np.array([0.9, 4.0, 0.5, 2.0, 0.8, 3.0, 1.5, 1.9, 2.5])
# ratios=np.array([0.3])
t_size2=1500

dimZ = 20
HU_decoder = 400
HU_encoder = HU_decoder
dimZ2 = dimZ
HU_decoder2 = HU_decoder
HU_encoder2 = HU_decoder2

batch_size = 100
L = 1
learning_rate = 0.01
sig=0.05
# maxiter=10
maxiter=2000

In [14]:
for ratio in tqdm(ratios):
    t_size1=int(t_size2/ratio)
    print(f"{t_size1 = }, {t_size2 = }")

    config={
        "samp_size": samp_size,
        "runs": runs,
        "ratio": ratio,
        "t_size2": t_size2,
        "t_size1": t_size1,
        "dimZ": dimZ,
        "HU_decoder": HU_decoder,
        "HU_encoder": HU_encoder,
        "dimZ2": dimZ2,
        "HU_decoder2": HU_decoder2,
        "HU_encoder2": HU_encoder2,
        "batch_size": batch_size,
        "L": L,
        "learning_rate": learning_rate,
        "sig": sig,
        "maxiter": maxiter
    }
    print(config)

    results = {
        "pvalue_win1" : [],
        "tstat_win1": [],
        "sigma_win1": [],
        "pvalue_win2" : [],
        "tstat_win2": [],
        "sigma_win2": [],
        "n_wins_1": 0,
        "n_wins_2": 0,
        "n_draws": 0
    }

    #dataset
    set1,set2 = train_test_split(range(data.shape[0]), train_size=t_size1+t_size2)
    data1 = data[set1[0:t_size1],:]
    data2 = data[set1[t_size1:t_size2+t_size1],:]
    set2=set2[0:samp_size]
    data_holdout=data[set2,:]
    print(f"{len(data1) = }, {len(data2) = }, {len(data_holdout) = }")

    #model initialization
    [N1,dimX] = data1.shape
    [N2,dimX] = data2.shape
    encoder1 = VariationalAutoencoder.VA(HU_decoder, HU_encoder, dimX, dimZ, batch_size, L, learning_rate, continous=False)
    encoder2 = VariationalAutoencoder.VA(HU_decoder2, HU_encoder2, dimX, dimZ2, batch_size, L, learning_rate, continous=False)
    encoder1.createGradientFunctions()
    encoder2.createGradientFunctions()
    encoder1.initParams()
    encoder2.initParams()

    begin = time.time()
    testlowerbound1=testlowerbound2=-np.Inf

    #training
    run = wandb.init(name=f"VAE comparizon, ratio={ratio}", project="Maximum mean discrepancy test", config=config)

    wandb.define_metric("vae1_step")
    wandb.define_metric("lb_train_1", step_metric="vae1_step")
    wandb.define_metric("lb_test_1", step_metric="vae1_step")

    for j in tqdm(range(maxiter)):
        encoder1.iterate(data1)
        if j%1 == 0:
            oldlower=testlowerbound1
            train_lower1=encoder1.getLowerBound(data1)
            testlowerbound1 = encoder1.getLowerBound(data_holdout)
            lbtrain = train_lower1/float(N1)
            lbtest = testlowerbound1/samp_size

            if(verbose):
                print(f"Encoder 1 Iteration {j}| lower bound train = {lbtrain} |lower bound test 1 = {lbtest}")
            wandb.log({"vae1_step": j, "lb_train_1": lbtrain, "lb_test_1": lbtest})

            if(oldlower>=testlowerbound1): break
            best_encoder1=copy.deepcopy(encoder1)

    encoder1=best_encoder1

    wandb.define_metric("vae2_step")
    wandb.define_metric("lb_train_2", step_metric="vae2_step")
    wandb.define_metric("lb_test_2", step_metric="vae2_step")

    for j in tqdm(range(maxiter)):
        encoder2.iterate(data2)

        if j%1 == 0:
            oldlower=testlowerbound2
            train_lower2=encoder2.getLowerBound(data2)
            testlowerbound2 = encoder2.getLowerBound(data_holdout)
            lbtrain = train_lower2/float(N2)
            lbtest = testlowerbound2/samp_size

            if(verbose):
                print("Encoder 2 Iteration %d| lower bound train = %.2f |lower bound test 1= %.2f"
                    % (j, lbtrain, lbtest))
            wandb.log({"vae2_step": j, "lb_train_2": lbtrain, "lb_test_2": lbtest})

            if(oldlower>testlowerbound2):
                break
            best_encoder2=copy.deepcopy(encoder2)

    encoder2=best_encoder2

    end=time.time()

    #test
    for _ in tqdm(range(runs)):
        samples1=encoder1.sample(N=samp_size)
        samples2=encoder2.sample(N=samp_size)

        pvalue_win1, tstat_win1, sigma_win1, MMDXY_win1, MMDXZ_win1 = MMD_3_Sample_Test(data_holdout, samples1, samples2, computeMMDs=False)
        print("win1: pvalue: %.2f"%(pvalue_win1))
        results["pvalue_win1"].append(pvalue_win1)
        results["tstat_win1"].append(tstat_win1)
        results["sigma_win1"].append(sigma_win1)
        pvalue_win2, tstat_win2, sigma_win2, MMDXY_win2, MMDXZ_win2 = MMD_3_Sample_Test(data_holdout, samples2, samples1, computeMMDs=False)
        print("win2: pvalue: %.2f"%(pvalue_win2))
        results["pvalue_win2"].append(pvalue_win2)
        results["tstat_win2"].append(tstat_win2)
        results["sigma_win2"].append(sigma_win2)

        alpha = 0.05
        if pvalue_win1 <= alpha and pvalue_win2 > alpha:
            results["n_wins_2"] += 1
        elif pvalue_win1 > alpha and pvalue_win2 <= alpha:
            results["n_wins_1"] += 1
        else:
            results["n_draws"] += 1
    print("n_wins_1:", results["n_wins_1"], ", n_wins_2:", results["n_wins_2"], ", n_draws:", results["n_draws"])

    data1_enc=encoder1.encode(x_valid)
    data2_enc=encoder2.encode(x_valid)
    test1_enc=encoder1.encode(x_test)
    test2_enc=encoder2.encode(x_test)

    LogReg_VAE = linear_model.LogisticRegression(max_iter=1000)
    LogReg_VAE.fit(data1_enc,t_valid)
    vae1_score = LogReg_VAE.score(test1_enc, t_test)
    LogReg_VAE = linear_model.LogisticRegression(max_iter=1000)
    LogReg_VAE.fit(data2_enc,t_valid)
    vae2_score = LogReg_VAE.score(test2_enc, t_test)
    print("Accuracy 1:%.2f Accuracy2:%.2f"%(vae1_score,vae2_score))

    wandb.define_metric("ratio")
    wandb.define_metric("n_wins_1", step_metric="ratio")
    wandb.define_metric("n_wins_2", step_metric="ratio")
    wandb.define_metric("n_draws", step_metric="ratio")
    wandb.define_metric("accuracy_1", step_metric="ratio")
    wandb.define_metric("accuracy_2", step_metric="ratio")
    wandb.log({
        "ratio": ratio,
        "n_wins_1": results["n_wins_1"],
        "n_wins_2": results["n_wins_2"],
        "n_draws": results["n_draws"],
        "accuracy_1": vae1_score,
        "accuracy_2": vae2_score
    })

    wandb.finish()

    with open(f"log_ratio_{ratio}.json", "w") as f:
        json.dump(results, f, indent=4)

  0%|          | 0/9 [00:00<?, ?it/s]

t_size1 = 1666, t_size2 = 1500
{'samp_size': 5000, 'runs': 10, 'ratio': 0.9, 't_size2': 1500, 't_size1': 1666, 'dimZ': 20, 'HU_decoder': 400, 'HU_encoder': 400, 'dimZ2': 20, 'HU_decoder2': 400, 'HU_encoder2': 400, 'batch_size': 100, 'L': 1, 'learning_rate': 0.01, 'sig': 0.05, 'maxiter': 2000}
len(data1) = 1666, len(data2) = 1500, len(data_holdout) = 5000


wandb: Currently logged in as: dmitriykornilov (dmitriykornilov_team). Use `wandb login --relogin` to force relogin


  0%|          | 0/2000 [00:00<?, ?it/s]

Encoder 1 Iteration 0| lower bound train = -0.15354781082648009 |lower bound test 1 = -0.05181105068531999
Encoder 1 Iteration 1| lower bound train = -0.14653817523430535 |lower bound test 1 = -0.04921831168012027
Encoder 1 Iteration 2| lower bound train = -0.1423527569751169 |lower bound test 1 = -0.04787365627152285
Encoder 1 Iteration 3| lower bound train = -0.15948451905531116 |lower bound test 1 = -0.053426271255734535


...

  0%|          | 0/10 [00:00<?, ?it/s]

win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
win1: pvalue: 0.00
win2: pvalue: 1.00
n_wins_1: 0 , n_wins_2: 10 , n_draws: 0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy 1:0.91 Accuracy2:0.91


accuracy_1,▁
accuracy_2,▁
lb_test_1,▁▇█████
lb_test_2,▁▄▅▇█▅
lb_train_1,▁▇█████
lb_train_2,▁▄▅▇█▅
n_draws,▁
n_wins_1,▁
n_wins_2,▁
ratio,▁
vae1_step,▁▂▃▅▆▇█


# Link to WandB
https://wandb.ai/dmitriykornilov_team/Maximum%20mean%20discrepancy%20test/workspace?workspace=user-dmitriykornilov


In [14]:
v